# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weatherpy_df = pd.read_csv('../output_data/cities.csv')
weatherpy_df.head(20)


,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,jamestown,42.0970,-79.2353,33.80,47,1,6.91,US,1612481333
1,airai,-8.9266,125.4092,62.98,100,100,3.83,TL,1612481396
2,verkh-chebula,56.0308,87.6218,5.22,94,98,8.61,RU,1612481484
3,sarangani,5.4033,125.4636,81.84,78,77,19.10,PH,1612481418
4,hobart,-42.8794,147.3294,71.60,78,75,5.75,AU,1612481310
5,arys,42.4252,68.7950,35.60,75,0,8.95,KZ,1612481485
6,mataura,-46.1927,168.8643,68.94,47,51,3.42,NZ,1612481406
7,tuktoyaktuk,69.4541,-133.0374,-34.60,74,75,12.66,CA,1612481409
8,moses lake,47.1301,-119.2781,51.80,43,1,10.36,US,1612481485
9,rikitea,-23.1203,-134.9692,78.85,68,97,10.94,PF,1612481114


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
